<a href="https://colab.research.google.com/github/KarelGans/Playstore-Comment-Sentiment-Analysis/blob/main/Proyek_Analisis_Sentimen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis

## Import Library

In [93]:
!pip install google-play-scraper

# Mengimpor pustaka google_play_scraper untuk mengakses ulasan dan informasi aplikasi dari Google Play Store.
from google_play_scraper import app, reviews, Sort, reviews_all

import pandas as pd  # Pandas untuk manipulasi dan analisis data
pd.options.mode.chained_assignment = None  # Menonaktifkan peringatan chaining
import numpy as np  # NumPy untuk komputasi numerik
seed = 0
np.random.seed(seed)  # Mengatur seed untuk reproduktibilitas
import matplotlib.pyplot as plt  # Matplotlib untuk visualisasi data
import seaborn as sns  # Seaborn untuk visualisasi data statistik, mengatur gaya visualisasi
from sklearn.metrics import accuracy_score

import datetime as dt  # Manipulasi data waktu dan tanggal
import re  # Modul untuk bekerja dengan ekspresi reguler
import string  # Berisi konstanta string, seperti tanda baca
from nltk.tokenize import word_tokenize  # Tokenisasi teks
from nltk.corpus import stopwords  # Daftar kata-kata berhenti dalam teks

!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory  # Stemming (penghilangan imbuhan kata) dalam bahasa Indonesia
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory  # Menghapus kata-kata berhenti dalam bahasa Indonesia

from wordcloud import WordCloud  # Membuat visualisasi berbentuk awan kata (word cloud) dari teks

import nltk  # Import pustaka NLTK (Natural Language Toolkit).
nltk.download('punkt')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stopwords) dalam berbagai bahasa.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Data Scraping

In [94]:
# Mengimpor pustaka google_play_scraper untuk mengakses ulasan dan informasi aplikasi dari Google Play Store.
from google_play_scraper import app, reviews_all, Sort

# Mengambil semua ulasan dari aplikasi dengan ID 'com.byu.id' di Google Play Store.
# Proses scraping mungkin memerlukan beberapa saat tergantung pada jumlah ulasan yang ada.
scrapreview = reviews_all(
    'com.telkomsel.telkomselcm',          # ID aplikasi
    lang='id',             # Bahasa ulasan (default: 'en')
    country='id',          # Negara (default: 'us')
    sort=Sort.MOST_RELEVANT, # Urutan ulasan (default: Sort.MOST_RELEVANT)
    count=3000             # Jumlah maksimum ulasan yang ingin diambil
)

In [95]:
# Menyimpan ulasan dalam file CSV
import csv

with open('ulasan_aplikasi.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Review'])  # Menulis header kolom
    for review in scrapreview:
        writer.writerow([review['content']])  # Menulis konten ulasan ke dalam file CSV

## Load Dataset

In [96]:
app_reviews_df = pd.DataFrame(scrapreview)
app_reviews_df.shape
app_reviews_df.head()
app_reviews_df.to_csv('ulasan_aplikasi.csv', index=False)

# Membuat DataFrame dari hasil scrapreview
app_reviews_df = pd.DataFrame(scrapreview)

# Menghitung jumlah baris dan kolom dalam DataFrame
jumlah_ulasan, jumlah_kolom = app_reviews_df.shape

# Menampilkan lima baris pertama dari DataFrame app_reviews_df
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,c243db1b-429d-49c4-8b0e-e2fde2931a21,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Dulu enak harga sama kualitas bagus, lah sekar...",1,0,6.1.0,2025-01-21 14:12:19,"Maaf ya, Kak Utep. Untuk kendala tidak mendapa...",2021-09-09 10:49:56,6.1.0
1,12cba177-9114-49b3-ac4e-577ea77c8b94,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,MyTelkomsel bisa dipake buat transaksi apapun ...,5,0,8.7.0,2025-03-30 09:23:24,None,NaT,8.7.0
2,15fc0c54-967b-4007-a0fd-ceb963ef0a7c,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,saya kasih bintang 3 karena masih banyak bug. ...,3,0,8.7.0,2025-03-30 14:53:05,None,NaT,8.7.0
3,a483e675-2e6f-4d1e-84d4-56708ca51767,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,saya naikin ke bintang 3 karna jaringan mulai ...,3,32,8.7.0,2025-03-28 12:06:51,Hai Kak Paulus. Mohon maaf untuk ketidaknyaman...,2024-11-08 01:26:27,8.7.0
4,dd2e0053-2b28-44b9-8a39-9e50172e85bb,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,bagus aplikasi terutama ada paket data exstra ...,5,0,8.7.0,2025-03-29 19:49:48,None,NaT,8.7.0


In [97]:
# Menampilkan informasi tentang DataFrame app_reviews_df
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108000 entries, 0 to 107999
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reviewId              108000 non-null  object        
 1   userName              108000 non-null  object        
 2   userImage             108000 non-null  object        
 3   content               108000 non-null  object        
 4   score                 108000 non-null  int64         
 5   thumbsUpCount         108000 non-null  int64         
 6   reviewCreatedVersion  90047 non-null   object        
 7   at                    108000 non-null  datetime64[ns]
 8   replyContent          55509 non-null   object        
 9   repliedAt             55509 non-null   datetime64[ns]
 10  appVersion            90047 non-null   object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 9.1+ MB


In [98]:
# Membuat DataFrame baru (clean_df) dengan menghapus baris yang memiliki nilai yang hilang (NaN) dari app_reviews_df
clean_df = app_reviews_df.dropna()

In [99]:
# Menghapus baris duplikat dari DataFrame clean_df
clean_df = clean_df.drop_duplicates()

# Menghitung jumlah baris dan kolom dalam DataFrame clean_df setelah menghapus duplikat
jumlah_ulasan_setelah_hapus_duplikat, jumlah_kolom_setelah_hapus_duplikat = clean_df.shape

## Text Preprocessing

In [100]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka

    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text

def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text

def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Memecah teks menjadi daftar kata
    words = text.split()

    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]

    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

In [101]:
def load_slang_dict(filename):
    """
    Converts a tab-separated text file of slang-formal word pairs into a Python dictionary

    Args:
        filename (str): Path to the text file

    Returns:
        dict: Dictionary with slang terms as keys and formal equivalents as values
    """
    slang_dict = {}

    with open(filename, 'r') as file:
        for line in file:
            # Remove whitespace and skip empty lines
            line = line.strip()
            if not line:
                continue

            # Split on tab character
            parts = line.split('\t')

            # Ensure we have exactly 2 elements (slang and formal)
            if len(parts) == 2:
                slang, formal = parts
                slang_dict[slang] = formal

    return slang_dict

slangwords = load_slang_dict("kbba.txt")
print(slangwords)

{'7an': 'tujuan', '@': 'di', 'ababil': 'abg labil', 'abis': 'habis', 'acc': 'accord', 'ad': 'ada', 'adlah': 'adalah', 'adlh': 'adalah', 'adoh': 'aduh', 'afaik': 'as far as i know', 'aha': 'tertawa', 'ahaha': 'haha', 'aing': 'saya', 'aj': 'saja', 'aja': 'saja', 'ajep-ajep': 'dunia gemerlap', 'ajj': 'saja', 'ak': 'saya', 'aka': 'dikenal juga sebagai', 'akika': 'aku', 'akko': 'aku', 'akkoh': 'aku', 'akku': 'aku', 'akuwh': 'aku', 'akyu': 'aku', 'alay': 'norak', 'aljasa': 'asal jadi saja', 'alow': 'halo', 'ama': 'sama', 'ambilin': 'ambilkan', 'ambl': 'ambil', 'ancur': 'hancur', 'anjir': 'anjing', 'anjrit': 'anjing', 'ank': 'anak', 'anter': 'antar', 'ap': 'apa', 'ap2': 'apa-apa', 'apaan': 'apa', 'apasih': 'apa sih', 'ape': 'apa', 'apes': 'sial', 'aplot': 'unggah', 'aps': 'apa', 'apva': 'apa', 'aq': 'saya', 'aqu': 'aku', 'aquwh': 'aku', 'asap': 'sesegera mungkin', 'asbun': 'asal bunyi', 'aseek': 'asyik', 'aseekk': 'asyik', 'asek': 'asyik', 'asekk': 'asyik', 'aseknya': 'asyiknya', 'asem': 'asa

In [102]:
#slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal"} #perlu cari dict slang karena dicoding kasih seuprit
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [103]:
import nltk
nltk.download('punkt_tab')
# Membersihkan teks dan menyimpannya di kolom 'text_clean'
clean_df['text_clean'] = clean_df['content'].apply(cleaningText)

# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)

# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)

# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)

# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [104]:
# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
clean_df['text_akhir'] = clean_df['text_stopword'].apply(toSentence)

In [105]:
clean_df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir
0,c243db1b-429d-49c4-8b0e-e2fde2931a21,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Dulu enak harga sama kualitas bagus, lah sekar...",1,0,6.1.0,2025-01-21 14:12:19,"Maaf ya, Kak Utep. Untuk kendala tidak mendapa...",2021-09-09 10:49:56,6.1.0,Dulu enak harga sama kualitas bagus lah sekara...,dulu enak harga sama kualitas bagus lah sekara...,dulu enak harga sama kualitas bagus lah sekara...,"[dulu, enak, harga, sama, kualitas, bagus, lah...","[enak, harga, kualitas, bagus, sinyal, penuh, ...",enak harga kualitas bagus sinyal penuh kualita...
3,a483e675-2e6f-4d1e-84d4-56708ca51767,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,saya naikin ke bintang 3 karna jaringan mulai ...,3,32,8.7.0,2025-03-28 12:06:51,Hai Kak Paulus. Mohon maaf untuk ketidaknyaman...,2024-11-08 01:26:27,8.7.0,saya naikin ke bintang karna jaringan mulai m...,saya naikin ke bintang karna jaringan mulai m...,saya naikin ke bintang karena jaringan mulai m...,"[saya, naikin, ke, bintang, karena, jaringan, ...","[naikin, bintang, jaringan, membaik, msih, suk...",naikin bintang jaringan membaik msih suka lamb...
8,6b60ee73-f92a-471b-a70a-a50828ec6723,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"setiap mau ngecek kuota, ga pernah update, loa...",1,253,8.7.0,2025-03-25 20:13:55,Hai Kak Abid. Mohon maaf untuk ketidaknyamanan...,2025-03-26 02:46:40,8.7.0,setiap mau ngecek kuota ga pernah update loadi...,setiap mau ngecek kuota ga pernah update loadi...,setiap mau ngecek kuota tidak pernah update lo...,"[setiap, mau, ngecek, kuota, tidak, pernah, up...","[ngecek, kuota, update, loading, melulu, jarin...",ngecek kuota update loading melulu jaringan ba...
9,66b56ebe-8fce-4300-8381-ee5c84d0e4a7,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"aplikasi lemot, padahal udah update tapi tetep...",1,60,8.7.0,2025-03-28 06:45:03,"Hai Kak Chandra, Mohon maaf untuk ketidaknyama...",2025-03-28 07:10:25,8.7.0,aplikasi lemot padahal udah update tapi tetep ...,aplikasi lemot padahal udah update tapi tetep ...,aplikasi lambat padahal sudah update tapi tete...,"[aplikasi, lambat, padahal, sudah, update, tap...","[aplikasi, lambat, update, tetep, masuk, tolon...",aplikasi lambat update tetep masuk tolong dipe...
11,4aee6a0d-9e29-442e-b2de-af752fb84225,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,dari 2 tahun aplikasi myTelkomsel nomor kartu ...,1,1,8.7.0,2025-03-30 02:54:14,"Hai Kak Aldimenur, Mohon maaf untuk ketidaknya...",2025-03-30 03:43:14,8.7.0,dari tahun aplikasi myTelkomsel nomor kartu H...,dari tahun aplikasi mytelkomsel nomor kartu h...,dari tahun aplikasi mytelkomsel nomor kartu ha...,"[dari, tahun, aplikasi, mytelkomsel, nomor, ka...","[aplikasi, mytelkomsel, nomor, kartu, halo, er...",aplikasi mytelkomsel nomor kartu halo error lo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107989,19588d51-275f-4035-b449-653195bc60bb,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Ini gmna si...beli paket katanya sudah termasu...,2,0,6.4.1,2021-11-25 02:08:25,"Maaf ya, Kak Ceka. Jika ada pertanyaan atau ke...",2021-11-25 02:14:55,6.4.1,Ini gmna sibeli paket katanya sudah termasuk l...,ini gmna sibeli paket katanya sudah termasuk l...,ini gmna sibeli paket katanya sudah termasuk l...,"[ini, gmna, sibeli, paket, katanya, sudah, ter...","[gmna, sibeli, paket, langgana, disney, masuk,...",gmna sibeli paket langgana disney masuk disney...
107990,ff610052-aa1f-417a-b2a4-d86651d41d19,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Harga paketnya makin lama makin mahal gak sepe...,1,0,5.12.0,2021-05-28 03:05:10,"Maaf ya, Kak Gabriella. Untuk keluhan aplikasi...",2021-05-28 04:16:00,5.12.0,Harga paketnya makin lama makin mahal ga

## Pelabelan

In [106]:
import csv
import requests
from io import StringIO

# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")

# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

In [107]:
# Fungsi untuk menentukan polaritas sentimen dari tweet

def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:

    score = 0
    # Inisialisasi skor sentimen ke 0

    for word in text:
        # Mengulangi setiap kata dalam teks

        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen

    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)

        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen

    polarity=''
    # Inisialisasi variabel polaritas

    if (score >= 0):
        polarity = 'positive'
        # Jika skor sentimen lebih besar atau sama dengan 0, maka polaritas adalah positif
    elif (score < 0):
        polarity = 'negative'
        # Jika skor sentimen kurang dari 0, maka polaritas adalah negatif

    # else:
    #     polarity = 'neutral'
    # Ini adalah bagian yang bisa digunakan untuk menentukan polaritas netral jika diperlukan

    return score, polarity
    # Mengembalikan skor sentimen dan polaritas teks

In [108]:
results = clean_df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_df['polarity_score'] = results[0]
clean_df['polarity'] = results[1]
print(clean_df['polarity'].value_counts())

polarity
negative    29734
positive    16452
Name: count, dtype: int64


## Data Splitting dan Ekstraksi Fitur dengan TF-IDF

In [109]:
clean_df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir,polarity_score,polarity
0,c243db1b-429d-49c4-8b0e-e2fde2931a21,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Dulu enak harga sama kualitas bagus, lah sekar...",1,0,6.1.0,2025-01-21 14:12:19,"Maaf ya, Kak Utep. Untuk kendala tidak mendapa...",2021-09-09 10:49:56,6.1.0,Dulu enak harga sama kualitas bagus lah sekara...,dulu enak harga sama kualitas bagus lah sekara...,dulu enak harga sama kualitas bagus lah sekara...,"[dulu, enak, harga, sama, kualitas, bagus, lah...","[enak, harga, kualitas, bagus, sinyal, penuh, ...",enak harga kualitas bagus sinyal penuh kualita...,2,positive
3,a483e675-2e6f-4d1e-84d4-56708ca51767,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,saya naikin ke bintang 3 karna jaringan mulai ...,3,32,8.7.0,2025-03-28 12:06:51,Hai Kak Paulus. Mohon maaf untuk ketidaknyaman...,2024-11-08 01:26:27,8.7.0,saya naikin ke bintang karna jaringan mulai m...,saya naikin ke bintang karna jaringan mulai m...,saya naikin ke bintang karena jaringan mulai m...,"[saya, naikin, ke, bintang, karena, jaringan, ...","[naikin, bintang, jaringan, membaik, msih, suk...",naikin bintang jaringan membaik msih suka lamb...,-14,negative
8,6b60ee73-f92a-471b-a70a-a50828ec6723,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"setiap mau ngecek kuota, ga pernah update, loa...",1,253,8.7.0,2025-03-25 20:13:55,Hai Kak Abid. Mohon maaf untuk ketidaknyamanan...,2025-03-26 02:46:40,8.7.0,setiap mau ngecek kuota ga pernah update loadi...,setiap mau ngecek kuota ga pernah update loadi...,setiap mau ngecek kuota tidak pernah update lo...,"[setiap, mau, ngecek, kuota, tidak, pernah, up...","[ngecek, kuota, update, loading, melulu, jarin...",ngecek kuota update loading melulu jaringan ba...,-20,negative
9,66b56ebe-8fce-4300-8381-ee5c84d0e4a7,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"aplikasi lemot, padahal udah update tapi tetep...",1,60,8.7.0,2025-03-28 06:45:03,"Hai Kak Chandra, Mohon maaf untuk ketidaknyama...",2025-03-28 07:10:25,8.7.0,aplikasi lemot padahal udah update tapi tetep ...,aplikasi lemot padahal udah update tapi tetep ...,aplikasi lambat padahal sudah update tapi tete...,"[aplikasi, lambat, padahal, sudah, update, tap...","[aplikasi, lambat, update, tetep, masuk, tolon...",aplikasi lambat update tetep masuk tolong dipe...,-7,negative
11,4aee6a0d-9e29-442e-b2de-af752fb84225,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,dari 2 tahun aplikasi myTelkomsel nomor kartu ...,1,1,8.7.0,2025-03-30 02:54:14,"Hai Kak Aldimenur, Mohon maaf untuk ketidaknya...",2025-03-30 03:43:14,8.7.0,dari tahun aplikasi myTelkomsel nomor kartu H...,dari tahun aplikasi mytelkomsel nomor kartu h...,dari tahun aplikasi mytelkomsel nomor kartu ha...,"[dari, tahun, aplikasi, mytelkomsel, nomor, ka...","[aplikasi, mytelkomsel, nomor, kartu, halo, er...",aplikasi mytelkomsel nomor kartu halo error lo...,-11,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107989,19588d51-275f-4035-b449-653195bc60bb,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Ini gmna si...beli paket katanya sudah termasu...,2,0,6.4.1,2021-11-25 02:08:25,"Maaf ya, Kak Ceka. Jika ada pertanyaan atau ke...",2021-11-25 02:14:55,6.4.1,Ini gmna sibeli paket katanya sudah termasuk l...,ini gmna sibeli paket katanya sudah termasuk l...,ini gmna sibeli paket katanya sudah termasuk l...,"[ini, gmna, sibeli, paket, katanya, sudah, ter...","[gmna, sibeli, paket, langgana, disney, masuk,...",gmna sibeli paket langgana disney masuk disney...,-1,negative
107990,ff610052-aa1f-417a-b2a4-d86651d41d19,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Harga paketnya makin lama makin mahal gak sepe...,1,0,5.12.0,2021-05-28 03:05:10,"Maaf ya, Kak

In [110]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Pisahkan data menjadi fitur (tweet) dan label (sentimen)
X = clean_df['text_akhir']
y = clean_df['polarity']

# Ekstraksi fitur dengan TF-IDF
tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8 )
X_tfidf = tfidf.fit_transform(X)

# Konversi hasil ekstraksi fitur menjadi dataframe
features_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())

# Menampilkan hasil ekstraksi fitur
features_df

# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

## Modeling

### Naive

In [111]:
from sklearn.naive_bayes import BernoulliNB

# Membuat objek model Naive Bayes (Bernoulli Naive Bayes)
naive_bayes = BernoulliNB()

# Melatih model Naive Bayes pada data pelatihan
naive_bayes.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_nb = naive_bayes.predict(X_train.toarray())
y_pred_test_nb = naive_bayes.predict(X_test.toarray())

# Evaluasi akurasi model Naive Bayes
accuracy_train_nb = accuracy_score(y_pred_train_nb, y_train)
accuracy_test_nb = accuracy_score(y_pred_test_nb, y_test)

# Menampilkan akurasi
print('Naive Bayes - accuracy_train:', accuracy_train_nb)
print('Naive Bayes - accuracy_test:', accuracy_test_nb)

Naive Bayes - accuracy_train: 0.7775793006387355
Naive Bayes - accuracy_test: 0.7722450746914916
